In [6]:
# 池化层
# 积对位置敏感
#   - 检测垂直边缘: 1的像素位移导致0的输出
#   1 1 0 0 0       0 1 0 0
#   1 1 0 0 0   ->  0 1 0 0
#   1 1 0 0 0   ->  0 1 0 0
#   1 1 0 0 0       0 1 0 0
# 需要一定程度的平移不变性
#   - 照明, 物体位置, 比例, 外观等等因图像而异
# 也就是说如果拍摄时动一动, 就会改变, 所以我们需要平移不变性来稳定

# 二维最大池化层
#  - 返回滑动窗口中的最大值
#  0 1 2  (2x2 pooling)    4 5
#  3 4 5      ->           7 8
#  6 7 8   max(0,1,3,4)
# 能否解决之前的问题？
#   1 1 0 0 0   (卷积)   0 1 0 0   pooling  1 1 0
#   1 1 0 0 0    ->     0 1 0 0      ->    1 1 0
#   1 1 0 0 0    ->     0 1 0 0      ->    1 1 0
#   1 1 0 0 0           0 1 0 0            1 1 0
# 2x2 pooling 容忍一个像素的偏移

# 填充, 步幅和多个通道
# 池化层与卷积层类似, 都具有填充和步幅
# 没有可以学习的参数
# 在每个输入通道应用池化层以获得相应的输出通道
# 输出通道 = 输入通道

# 平均池化层
#  - 最大池化层: 每个窗口中最强的模式型号
#  - 平均池化层: 将最大池化层的"最大"操作替换为"平均"

# 总结
#  - 池化层返回窗口中最大或平均值
#  - 缓解卷积层对位置的敏感性 (通常作用在卷积层之后)
#  - 不同在与对每个通道数分别作用, 直接输出, 所以(输入通道数 = 输出通道数)
#  - 同样有窗口大小, 填充, 和步幅作为超参数

In [9]:
# 实现池化层的正向传播
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_w, p_h = pool_size # 得到池化层size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w +1)) # 计算输出的大小
    for i in range(Y.shape[0]):  # loop through height
        for j in range(Y.shape[1]): # loop through width
            if mode == 'max':
                Y[i, j] = X[i: i+p_h, j: j+p_w].max() # 取X中经过池化层后的最大值, 赋值到Y
            if mode == 'avg':
                Y[i, j] = X[i: i+p_h, j: j+p_w].mean() # 取X中经过池化层后的平均值, 赋值到Y
    return Y

In [10]:
# 验证二维池化层的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [11]:
# 验证平均汇聚层
pool2d(X, (2,2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

In [12]:
# 填充和步幅
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [13]:
# 深度学习框架中的步幅和池化窗口大小相同
pool2d = nn.MaxPool2d(3) # 注意size为3x3, 步幅为3
pool2d(X)

tensor([[[[10.]]]])

In [14]:
# 填充和步幅可以手动设定
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [15]:
# 设定一个任意大小的矩形池化窗口, 并分别设定填充和步幅的高度和宽度
pool2d = nn.MaxPool2d((2, 3), padding=(1,1), stride=(2,3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [16]:
# 池化层在每个通道上的单独运算
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [17]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])